In [1]:
import numpy as np


In [5]:
driver_behavior = []

In [7]:
# Randomly initiate driver behaviors

for i in range(0,10):
    attentive_db = np.random.binomial(1, 0.8)
    
    # We can add rules of if then for what could be combined and what could not
    if attentive_db == 0:
        drowsey_db = np.random.binomial(1, 0.01454)
        distracted_db = np.random.binomial(1, 0.04633)
        speeding_db = np.random.binomial(1, 0.02778)
        exceed_safe_speed_db = np.random.binomial(1, 0.00888)
    else:
        drowsey_db = 0
        distracted_db = 0
        speeding_db = 0
        exceed_safe_speed_db = 0
    
    # create a list or dictionary for the behavior that each driver exibits (1 means it has behavior, 0 means it does not)
    driver_behavior.append([attentive_db, drowsey_db, distracted_db, speeding_db, exceed_safe_speed_db])
    
    

In [8]:
driver_behavior # We may need to loop through a couple times to make sure at least one driver behavior is initiated

[[1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0]]

In [ ]:
# Based on driver behavior select driver impairment